# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier 



## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
df = pd.read_csv("/Users/aliait/Desktop/LoanStats_2019Q1.csv")
# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [51]:
# Create our features
X = df.drop(columns='loan_status')
X = pd.get_dummies(X)
X.head(2)

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1


In [7]:
# Create our target
y = df[["loan_status"]]

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [9]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk       68470
high_risk        347
dtype: int64

In [10]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train,X_test,y_train,y_test=train_test_split(X,y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [11]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [12]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [13]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners


### Balanced Random Forest Classifier

In [47]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)
rf_model = rf_model.fit(X_train_scaled, y_train)
 

In [48]:
# Calculated the balanced accuracy score
y_pred_rf = rf_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_rf)

0.6921031813809657

In [49]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_rf)

array([[   35,    56],
       [    7, 17107]])

In [50]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_rf, digits=5))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk    0.83333   0.38462   0.99959   0.52632   0.62005   0.36081        91
   low_risk    0.99674   0.99959   0.38462   0.99816   0.62005   0.40810     17114

avg / total    0.99587   0.99634   0.38787   0.99567   0.62005   0.40785     17205



In [30]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted_features=sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
sorted_features

[(0.0669978711370857, 'total_pymnt'),
 (0.06498557404320127, 'total_rec_int'),
 (0.06380912838474019, 'last_pymnt_amnt'),
 (0.059897974883809824, 'total_pymnt_inv'),
 (0.05936050351797263, 'total_rec_prncp'),
 (0.023600121048794276, 'out_prncp'),
 (0.02275095192849459, 'installment'),
 (0.022275483044748447, 'out_prncp_inv'),
 (0.01933216709962185, 'dti'),
 (0.018215576598892592, 'mo_sin_old_rev_tl_op'),
 (0.016216634540279218, 'total_rec_late_fee'),
 (0.01578811375472831, 'max_bal_bc'),
 (0.015568579702949448, 'bc_open_to_buy'),
 (0.015101608940190206, 'annual_inc'),
 (0.015074286465804592, 'bc_util'),
 (0.014885107366342253, 'mo_sin_old_il_acct'),
 (0.014621890488029666, 'tot_cur_bal'),
 (0.014388064237426506, 'total_bal_ex_mort'),
 (0.014244479320410645, 'tot_hi_cred_lim'),
 (0.014172022983503714, 'loan_amnt'),
 (0.013468882788139904, 'mths_since_recent_bc'),
 (0.01345867154155198, 'all_util'),
 (0.013261907169062946, 'avg_cur_bal'),
 (0.013102512251833159, 'mths_since_rcnt_il'),
 (

### Easy Ensemble Classifier

In [43]:
# Train the Classifier
eec_model = EasyEnsembleClassifier(n_estimators=100, random_state=15)
eec_model = eec_model.fit(X_train_scaled, y_train)


In [44]:
# Calculated the balanced accuracy score
y_pred_ecc = rf_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_ecc)

0.7085866978644821

In [42]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_ecc)

array([[   38,    53],
       [    7, 17107]])

In [39]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_ecc,digits=5))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk    0.84444   0.41758   0.99959   0.55882   0.64607   0.39312        91
   low_risk    0.99691   0.99959   0.41758   0.99825   0.64607   0.44171     17114

avg / total    0.99610   0.99651   0.42066   0.99593   0.64607   0.44145     17205



### Model Comparaison


    The EASY Ensemble Classifier has a better accuracy score, recall score and geo mean score

    The Top 3 features are: total_pymnt_inv, last_pymnt_amnt , total_rec_prncp
